### Using URLConfigurations to reprocess data with straxen

Note a fully guide showing how to use URL configurations for every correction will be on a separate notebook, this is just meant as a generic example

In [1]:
import strax
import straxen
import xedocs as xd
import numpy as np
import rframe
import pymongo
import matplotlib.pyplot as plt

Could not import editors, GUI not available.


In [2]:
straxen.print_versions('strax straxen rframe xedocs'.split())

,module,version,path,git
0,python,3.8.13,/opt/conda/envs/strax/bin/python,None
1,strax,1.4.0,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
2,straxen,2.0.3,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
3,rframe,0.2.1,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b
4,xedocs,0.2.8,/home/luissanchez25/.local/lib/python3.8/site-...,branch:relative_light_yield_schema | 3cd255b


All corrections in straxen have been changed to use URLConfigurations as these are more flexible and do not require us to have to change straxen every time we what to update the corrections. There are protocols in straxen and xedocs that take in a string URL and with this they are able to find and load the requested data.

In [3]:
url = './test_data/'

In [4]:
run_id = '047493'

In [5]:
st = straxen.contexts.xenonnt_online(output_folder = url, include_rucio_remote=True,download_heavy=True)

In [6]:
st_xd = straxen.contexts.xenonnt_online(output_folder = url, include_rucio_remote=True,download_heavy=True)

In [8]:
xd_db_stx = xd.straxen_db.utilix

In [9]:
xd_db_dev = xd.development_db.utilix

In [18]:
xd_db_dev.electron_lifetimes.find_one(version = 'test*', run_id = run_id)


        Xenon ElectronLifetime Document
        -------------------------------
 
        Category:      corrections
        Alias:         electron_lifetimes
        Index:         version=test*, time=2022-09-13 11:14:46.741000
        Values:        created_date=2023-01-24 17:20:42.436000, comments=giving a 10% decrease to Electron lifetime, reviews=[], value=7362379.160154001
        

In [28]:
st_xd.set_config({'elife':'xedocs://'
                  'electron_lifetimes'
                  '?version=test*&db=development_db&run_id=047493&attr=value'})

In [25]:
st.get_single_plugin(run_id, 'corrected_areas').elife

8180421.28906

In [29]:
st_xd.get_single_plugin(run_id, 'corrected_areas').elife

7362379.160154001

As we can see we have successfully changed the corrections of electron lifetime to a different value, now all we have to do is reprocess!

Only run the following lines if you have access to run 047493

In [ ]:
kr_ei_xd = st_xd.get_array(run_id, 'event_info')
kr_ei = st.get_array(run_id, 'event_info')

In [ ]:
plt.hist(kr_ei_xd['cs2'], bins = np.linspace(80,2000, 100), histtype = 'step', color = 'purple', label = 'elife 1% deviation')
plt.hist(kr_ei['cs2'], bins = np.linspace(80,2000, 100), histtype = 'step', color = 'orange', label = 'elife')
plt.xlabel('cS2')
plt.ylabel('count')
plt.legend()